In [ ]:
import datetime

import mariadb
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
path = 'data/users.csv'
df_user = pd.read_csv(path)
df_user

In [ ]:
path = 'data/problems.csv'
df_problem = pd.read_csv(path)
df_problem = df_problem.astype({})
df_problem = df_problem.astype({'subject': 'str', 'week': 'int8', 'EMF': 'str',
                                'problem': 'int16', 'score': 'float32', 'HML': 'str',
                                '1.00': 'str', '0.70': 'str', 
                                '0.40': 'str', '0.10': 'str',
                               })
df_problem.loc[:, '1.00'] = pd.to_datetime(df_problem.loc[:, '1.00'])
df_problem.loc[:, '0.70'] = pd.to_datetime(df_problem.loc[:, '0.70'])
df_problem.loc[:, '0.40'] = pd.to_datetime(df_problem.loc[:, '0.40'])
df_problem.loc[:, '0.10'] = pd.to_datetime(df_problem.loc[:, '0.10'])
df_problem

In [ ]:
import secret
conn = mariadb.connect(user=secret.user, password=secret.password,
                       host='127.0.0.1', port=3306)
cur = conn.cursor()

# 과목별 실습 주수

In [ ]:
print('과목별 사용자 수')
df_problem.groupby(by='subject').max()['week']

# 과목별 학생 수

In [ ]:
print('과목별 사용자 수')
for subject in df_user['subject'].unique():
    print(f'{subject:>12s} => {len(df_user[df_user["subject"] == subject]):>2d}')
print(f'총 학생 수: {len(df_user)}')

# 과목별 제출 수

In [ ]:
df = pd.DataFrame()
for subject in ['2019-25155']:
    for user in df_user['name']:
        for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
            try:
                cur.execute('''SELECT dj2019.judging.result, COUNT(*)
                               FROM dj2019.judging
                               INNER JOIN dj2019.submission ON dj2019.judging.submitid = dj2019.submission.submitid
                               INNER JOIN dj2019.team ON dj2019.submission.teamid = dj2019.team.teamid
                               WHERE dj2019.team.name LIKE ?
                                 AND dj2019.submission.probid = ?
                               GROUP BY dj2019.judging.result;''', (user, problem))
                for row in cur:
                    df = df.append({'subject': subject,
                                    'user': user,
                                    'problem': problem,
                                    'result': row[0],
                                    'count': row[1]}, ignore_index=True)
            except TypeError:
                continue
for subject in ['2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    for user in df_user['name']:
        for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
            try:
                cur.execute('''SELECT dj2020.judging.result, COUNT(*)
                               FROM dj2020.judging
                               INNER JOIN dj2020.submission ON dj2020.judging.submitid = dj2020.submission.submitid
                               INNER JOIN dj2020.team ON dj2020.submission.teamid = dj2020.team.teamid
                               WHERE dj2020.team.name LIKE ?
                                 AND dj2020.submission.probid = ?
                               GROUP BY dj2020.judging.result;''', (user, problem))
                for row in cur:
                    df = df.append({'subject': subject,
                                    'user': user,
                                    'problem': problem,
                                    'result': row[0],
                                    'count': row[1]}, ignore_index=True)
            except TypeError:
                continue
df.drop(columns='problem', inplace=True)
df = df.astype({'count': 'int32'})
df = df.groupby(by=['subject', 'result']).sum()
df

In [ ]:
df = df.groupby(by=['result']).sum()
df

In [ ]:
df/sum(df['count'])

In [ ]:
path = 'submissions_by_subject.pkl'
# df.to_pickle(path)
df = pd.read_pickle(path)

In [ ]:
labels = ['2019-25155', '2020-37777', '2020-25155', '2020-27401A', '2020-27401B']
errors = ['correct', 'wrong-answer', 'timelimit', 'memorylimit', 'run-error', 'compiler-error']
total = df.sum(level='subject').loc[labels, 'count'].values
fig = plt.figure(figsize=(8*1, 6*1))
fig.set_facecolor('white')
ax = fig.add_subplot()
stacked = np.zeros(len(labels), dtype=np.int32)
data = []
for label in labels:
    data.append(df.loc[(label, 'correct'), 'count'])
data = np.array(data, dtype=np.int32) / total
rect = ax.bar(x=labels, height=data, bottom=stacked, label='C')
ax.bar_label(rect, fmt='%0.2f', label_type='center')
stacked = stacked + data
data = []
for label in labels:
    data.append(df.loc[(label, 'wrong-answer'), 'count'] + df.loc[(label, 'no-output'), 'count'])
data = np.array(data, dtype=np.int32) / total
rect = ax.bar(x=labels, height=data, bottom=stacked, label='WA')
ax.bar_label(rect, fmt='%0.2f', label_type='center')
stacked = stacked + data
data = []
for label in labels:
    data.append(df.loc[(label, 'timelimit'), 'count'])
data = np.array(data, dtype=np.int32) / total
rect = ax.bar(x=labels, height=data, bottom=stacked, label='TLE')
ax.bar_label(rect, fmt='%0.2f', label_type='center')
stacked = stacked + data
# for label in labels:
#     data.append(df.loc[(label, 'memorylimit'), 'count'])
data = [0, 0, 0, 0, 0]
data = np.array(data, dtype=np.int32) / total
rect = ax.bar(x=labels, height=data, bottom=stacked, label='MLE')
# ax.bar_label(rect, fmt='%0.2f', label_type='center')
stacked = stacked + data
data = []
for label in labels:
    data.append(df.loc[(label, 'run-error'), 'count'])
data = np.array(data, dtype=np.int32) / total
rect = ax.bar(x=labels, height=data, bottom=stacked, label='RTE')
ax.bar_label(rect, fmt='%0.2f', label_type='center')
stacked = stacked + data
data = []
for label in labels:
    data.append(df.loc[(label, 'compiler-error'), 'count'])
data = np.array(data, dtype=np.int32) / total
rect = ax.bar(x=labels, height=data, bottom=stacked, label='CE')
ax.bar_label(rect, fmt='%0.2f', label_type='center')
stacked = stacked + data
ax.set_ylim((0, 1))
ax.tick_params(axis='x', labelsize='large')
ax.tick_params(axis='y', labelsize='large')
ax.set_xlabel('Subject', fontsize='x-large')
ax.set_ylabel('Count', fontsize='x-large')
_ = ax.legend(loc='lower right', ncol=2)
fig.savefig('submission_by_subject.png', bbox_inches='tight')

# 학생 성적 분류

In [ ]:
TZ_UTC = datetime.timezone(datetime.timedelta())
TZ_SEOUL = datetime.timezone(datetime.timedelta(hours=9))

In [ ]:
cnt = 0
# df = pd.DataFrame()
data = []
for subject in ['2019-25155']:
    for user in df_user.loc[df_user['subject'] == subject, 'name']:
        for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
            week = df_problem.loc[(df_problem['subject'] == subject) & (df_problem['problem'] == problem), 'week'].values[0]
            try:
                cur.execute('''SELECT MIN(dj2019.judging.starttime)
                               FROM dj2019.judging
                               INNER JOIN dj2019.submission ON dj2019.judging.submitid = dj2019.submission.submitid
                               INNER JOIN dj2019.team ON dj2019.submission.teamid = dj2019.team.teamid
                               WHERE dj2019.team.name LIKE ?
                                 AND dj2019.submission.probid = ?
                                 AND dj2019.judging.result LIKE 'correct';''', (user, problem))
                pass_time = datetime.datetime.fromtimestamp(float(cur.fetchone()[0]), tz=TZ_SEOUL)
                condition = (df_problem['subject'] == subject) & (df_problem['problem'] == problem)
                if pass_time <= df_problem.loc[condition, '1.00'].dt.to_pydatetime()[0]:
                    score = 1.0 * df_problem.loc[condition, 'score'].values[0]
                elif pass_time <= df_problem.loc[condition, '0.70'].dt.to_pydatetime()[0]:
                    score = 0.7 * df_problem.loc[condition, 'score'].values[0]
                elif pass_time <= df_problem.loc[condition, '0.40'].dt.to_pydatetime()[0]:
                    score = 0.4 * df_problem.loc[condition, 'score'].values[0]
                elif pass_time <= df_problem.loc[condition, '0.10'].dt.to_pydatetime()[0]:
                    score = 0.1 * df_problem.loc[condition, 'score'].values[0]
                else:
                    score = 0
            except TypeError:
                score = 0
            data.append({'subject': subject,
                         'user': user,
                         'week': week,
                         'problem': problem,
                         'score': score})
for subject in ['2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    for user in df_user.loc[df_user['subject'] == subject, 'name']:
        for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
            week = df_problem.loc[(df_problem['subject'] == subject) & (df_problem['problem'] == problem), 'week'].values[0]
            try:
                cur.execute('''SELECT MIN(dj2020.judging.starttime)
                               FROM dj2020.judging
                               INNER JOIN dj2020.submission ON dj2020.judging.submitid = dj2020.submission.submitid
                               INNER JOIN dj2020.team ON dj2020.submission.teamid = dj2020.team.teamid
                               WHERE dj2020.team.name LIKE ?
                                 AND dj2020.submission.probid = ?
                                 AND dj2020.judging.result LIKE 'correct';''', (user, problem))
                pass_time = datetime.datetime.fromtimestamp(float(cur.fetchone()[0]), tz=TZ_SEOUL)
                condition = (df_problem['subject'] == subject) & (df_problem['problem'] == problem)
                if pass_time <= df_problem.loc[condition, '1.00'].dt.to_pydatetime()[0]:
                    score = 1.0 * df_problem.loc[condition, 'score'].values[0]
                elif pass_time <= df_problem.loc[condition, '0.70'].dt.to_pydatetime()[0]:
                    score = 0.7 * df_problem.loc[condition, 'score'].values[0]
                elif pass_time <= df_problem.loc[condition, '0.40'].dt.to_pydatetime()[0]:
                    score = 0.4 * df_problem.loc[condition, 'score'].values[0]
                elif pass_time <= df_problem.loc[condition, '0.10'].dt.to_pydatetime()[0]:
                    score = 0.1 * df_problem.loc[condition, 'score'].values[0]
                else:
                    score = 0
            except TypeError:
                score = 0
            data.append({'subject': subject,
                         'user': user,
                         'week': week,
                         'problem': problem,
                         'score': score})
df = pd.DataFrame(data)
df

In [ ]:
path = 'score_by_problem.pkl'
# df.to_pickle(path)
df = pd.read_pickle(path)

In [ ]:
import math
import collections
import operator

result = pd.DataFrame()
for subject in ['2019-25155', '2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    users = len(df_user.loc[df_user['subject'] == subject])
    a = math.floor(users * 0.3)
    c = math.ceil(users * 0.3)
    b = users - (a+c)
    first = 3
    half = max(df_problem.loc[(df_problem['subject'] == subject) & (df_problem['EMF'] == 'M'), 'week'])
    final = max(df_problem.loc[(df_problem['subject'] == subject) & (df_problem['EMF'] == 'F'), 'week'])
    
    tdf = df.loc[(df['subject'] == subject) & (df['week'] <= first)].groupby(by=['subject', 'user']).sum()
    tdf.drop(columns=['week', 'problem'], inplace=True)
    score_table = list(collections.Counter(tdf.sort_values(by='score', ascending=False)['score']).items())
    score_table.sort(key=operator.itemgetter(0), reverse=True)
    cnt = a
    a_cut = 0
    for table in score_table:
        if cnt >= table[1]:
            cnt = cnt - table[1]
            a_cut = table[0]
        else:
            break
    cnt = c
    c_cut = 0
    for table in score_table[::-1]:
        if cnt >= table[1]:
            cnt = cnt - table[1]
            c_cut = table[0]
        else:
            break
    data = []
    for user in tdf[tdf['score'] >= a_cut].index.get_level_values('user').unique():
        data.append({'subject': subject,
                     'user': user,
                     'first': 'A'})
    for user in tdf[(c_cut < tdf['score']) & (tdf['score'] < a_cut)].index.get_level_values('user').unique():
        data.append({'subject': subject,
                     'user': user,
                     'first': 'B'})
    for user in tdf[tdf['score'] <= c_cut].index.get_level_values('user').unique():
        data.append({'subject': subject,
                     'user': user,
                     'first': 'C'})
    result = result.append(data, ignore_index=True)

    tdf = df.loc[(df['subject'] == subject) & (df['week'] <= half)].groupby(by=['subject', 'user']).sum()
    tdf.drop(columns=['week', 'problem'], inplace=True)
    score_table = list(collections.Counter(tdf.sort_values(by='score', ascending=False)['score']).items())
    score_table.sort(key=operator.itemgetter(0), reverse=True)
    cnt = a
    a_cut = 0
    for table in score_table:
        if cnt >= table[1]:
            cnt = cnt - table[1]
            a_cut = table[0]
        else:
            break
    cnt = c
    c_cut = 0
    for table in score_table[::-1]:
        if cnt >= table[1]:
            cnt = cnt - table[1]
            c_cut = table[0]
        else:
            break
    for user in tdf[tdf['score'] >= a_cut].index.get_level_values('user').unique():
        result.loc[(result['subject'] == subject) & (result['user'] == user), 'half'] = 'A'
    for user in tdf[(c_cut < tdf['score']) & (tdf['score'] < a_cut)].index.get_level_values('user').unique():
        result.loc[(result['subject'] == subject) & (result['user'] == user), 'half'] = 'B'
    for user in tdf[tdf['score'] <= c_cut].index.get_level_values('user').unique():
        result.loc[(result['subject'] == subject) & (result['user'] == user), 'half'] = 'C'

    tdf = df.loc[(df['subject'] == subject) & (df['week'] <= final)].groupby(by=['subject', 'user']).sum()
    tdf.drop(columns=['week', 'problem'], inplace=True)
    score_table = list(collections.Counter(tdf.sort_values(by='score', ascending=False)['score']).items())
    score_table.sort(key=operator.itemgetter(0), reverse=True)
    cnt = a
    a_cut = 0
    for table in score_table:
        if cnt >= table[1]:
            cnt = cnt - table[1]
            a_cut = table[0]
        else:
            break
    cnt = c
    c_cut = 0
    for table in score_table[::-1]:
        if cnt >= table[1]:
            cnt = cnt - table[1]
            c_cut = table[0]
        else:
            break
    for user in tdf[tdf['score'] >= a_cut].index.get_level_values('user').unique():
        result.loc[(result['subject'] == subject) & (result['user'] == user), 'final'] = 'A'
    for user in tdf[(c_cut < tdf['score']) & (tdf['score'] < a_cut)].index.get_level_values('user').unique():
        result.loc[(result['subject'] == subject) & (result['user'] == user), 'final'] = 'B'
    for user in tdf[tdf['score'] <= c_cut].index.get_level_values('user').unique():
        result.loc[(result['subject'] == subject) & (result['user'] == user), 'final'] = 'C'
rank = result
rank

In [ ]:
rank.groupby(by=['subject', 'first', 'half', 'final']).count()

In [ ]:
t[t.index.get_level_values('first') > t.index.get_level_values('final')]

In [ ]:
sum(t[t.index.get_level_values('first') > t.index.get_level_values('final')]['user'].values)

# 학생 답안 예측

In [ ]:
def parse_python(code):
    tables = {}
    table = {}
    whitespace = -1
    for line in code.split('\n'):
        level = len(line) - len(line.lstrip())
        if whitespace != level:
            table = tables.get(level, {})
            whitespace = level
        if 'for' in line or 'while' in line:
            whitespace = -1
        if '=' not in line:
            continue
        left = line.split('=')[0]
        left = left.replace('(', '')
        left = left.replace(')', '')
        left = left.replace(',', '')
        
        right = line.split('=')[1]
        if 'int' in right:
            typ = 'int'
        elif 'str' in right:
            typ = 'str'
        elif 'float' in right:
            typ = 'float'
        elif '[]' in right or 'list' in right:
            typ = 'list'
        elif '{}' in right or 'dict' in right:
            typ = 'dict'
        elif 'set' in right:
            typ = 'set'
        else:
            find = False
            for v in right.split(' '):
                if v in table.keys():
                    typ = table[v]
                    find = True
            if not find:
                typ = 'int'
        
        for v in left.split(' '):
            if '[' in v:
                v = v.split('[')[0]
            if len(v) == 0:
                continue
            table[v] = typ
        tables[level] = table

    results = []
    for level, table in sorted(list(tables.items())):
        result = {}
        for key, value in table.items():
            result[value] = result.get(value, 0) + 1
        results.append(result)

    return results


def parse_java(code):
    tables = {}
    table = {}
    loop = 0
    brackets = 0
    in_loop = False
    for line in code.split('\n'):
        if 'for' in line or 'while' in line:
            in_loop = True
        if in_loop:
            if '{' in line:
                loop = loop + 1
                in_loop = False
        if '}' in line:
            for c in line:
                if c == '}':
                    if brackets > 0:
                        brackets = brackets - 1
                    elif loop > 0:
                        loop = loop - 1
        if '=' not in line:
            continue
        if '=' in line and 'for' in line:
            continue
        lefts = line.split('=')[0]
        lefts = [l for l in lefts.split(' ') if len(l) != 0]
        if len(lefts) >= 2:
            name = lefts[-1]
            if '[' in name:
                name = name.split('[')[0]
            typ = lefts[-2].strip()
        elif len(lefts) == 1:
            name = lefts[0]
            find = False
            for table in tables.values():
                if name in tables.keys():
                    typ = table[v]
                    find = True
                    break
            right = line.split('=')[1]
            if not find:
                if 'int' in right:
                    typ = 'int'
                elif 'str' in right or 'sc' in right or 'scan' in right or 'br' in right or 'buf' in right:
                    typ = 'str'
                elif 'double' in right or 'float' in right:
                    typ = 'float'
                elif 'list' in right or 'stack' in right or 'queue' in right:
                    typ = 'list'
                elif 'map' in right:
                    typ = 'dict'
                elif 'set' in right:
                    typ = 'set'
                else:
                    typ = 'int'
        if '[' in typ:
            typ = typ.split('[')[0]
        if '<' in typ:
            typ = typ.split('<')[0]
        if '(' in typ:
            typ = typ.split('(')[1]
        table = tables.get(loop, {})
        table[name] = typ
        tables[loop] = table

    results = []
    for level, table in sorted(list(tables.items())):
        result = {}
        for key, value in table.items():
            result[value] = result.get(value, 0) + 1
        results.append(result)

    return results

In [ ]:
import math
import operator


def distance_codemap(a, b):
    value = 0
    for i in range(max(len(a), len(b))):
        if i < len(a):
            al = a[i]
        else:
            al = {}
        if i < len(b):
            bl = b[i]
        else:
            bl = {}
        keys = set(al.keys()) | set(bl.keys())
        for key in keys:
            value = value + distance_var(al, bl, key)
    value = math.sqrt(value)
    return value


def distance_var(a, b, v):
    va = a.get(v, 0)
    vb = b.get(v, 0)
    value = (va-vb)**2
    return value


def distance_var2(a, b, v):
    value = 0
    for i in range(max(len(a), len(b))):
        if i < len(a):
            al = a[i]
        else:
            al = {}
        if i < len(b):
            bl = b[i]
        else:
            bl = {}
        value = value + distance_var(al, bl, v)
    value = math.sqrt(value)
    return value


def recommend_codemap(a, b): # a is answer
    table = []
    for index, codemap in enumerate(a):
        table.append((distance_codemap(codemap, b), index))
    table.sort(key=operator.itemgetter(0), reverse=False)
    index = table[0][1]
    return index, a[index]


def recommend_var(a, b): # a is answer
    table = {}
    for i, ac in enumerate(a):
        if i < len(a):
            al = a[i]
        else:
            al = {}
        if i < len(b):
            bl = b[i]
        else:
            bl = {}
        keys = set(al.keys())
        for key in keys:
            value = distance_var(al, bl, key)
            table[key] = math.sqrt(value)
    cand = sorted(list(table.items()), key=operator.itemgetter(1), reverse=True)
    return cand[0]

In [ ]:
cnt = 0
data = []

for subject in ['2019-25155']:
    answers = {}
    wrongs = {}
    for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
        try:
            cur.execute('''SELECT dj2019.judging.result, dj2019.submission.langid, dj2019.submission_file.sourcecode,
                                  dj2019.team.name
                           FROM dj2019.submission_file
                           INNER JOIN dj2019.submission ON dj2019.submission_file.submitid = dj2019.submission.submitid
                           INNER JOIN dj2019.judging ON dj2019.submission_file.submitid = dj2019.judging.submitid
                           INNER JOIN dj2019.team ON dj2019.submission.teamid = dj2019.team.teamid
                           WHERE dj2019.submission.probid = ?
                           ORDER BY dj2019.judging.starttime ASC;''', (problem,))
            for row in cur:
                result = row[0]
                lang = row[1]
                try:
                    code = row[2].decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        code = row[2].decode('euc-kr')
                    except UnicodeDecodeError:
                        code = code
                user = row[3]
                if 'py' not in lang and 'java' not in lang:
                    continue

                if result == 'correct':
                    answer4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    answer4lang[problem] = answer
                    answers[lang] = answer4lang
                    
                    if user in wrongs.keys():
                        wrongs4team = wrongs[user]
                        if problem in wrongs4team.keys():
                            var, var_dis, wrong_codemap = wrongs4team[problem]
                            v = distance_var2(codemap, wrong_codemap, var)
                            data.append({'subject': subject,
                                         'user': user,
                                         'problem': problem,
                                         'diff': abs(var_dis-v)})
                        if problem in wrongs4team.keys():
                            wrongs4team.pop(problem)
                    if user in wrongs.keys() and len(wrongs[user]) == 0:
                        wrongs.pop(user)
                elif result == 'wrong-answer':
                    answers4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                    if len(answer) > 0:
                        index, answer_codemap = recommend_codemap(answer, codemap)
                        distance = distance_codemap(answer_codemap, codemap)
                        var, var_dis = recommend_var(answer_codemap, codemap)
                        wrongs4team = wrongs.get(user, {})
                        if problem not in wrongs4team.keys():
                            wrongs4team[problem] = (var, var_dis, codemap)
                        wrongs[user] = wrongs4team
        except TypeError:
            continue
for subject in ['2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    answers = {}
    wrongs = {}
    for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
        try:
            cur.execute('''SELECT dj2020.judging.result, dj2020.submission.langid, dj2020.submission_file.sourcecode,
                                  dj2020.team.name
                           FROM dj2020.submission_file
                           INNER JOIN dj2020.submission ON dj2020.submission_file.submitid = dj2020.submission.submitid
                           INNER JOIN dj2020.judging ON dj2020.submission_file.submitid = dj2020.judging.submitid
                           INNER JOIN dj2020.team ON dj2020.submission.teamid = dj2020.team.teamid
                           WHERE dj2020.submission.probid = ?
                           ORDER BY dj2020.judging.starttime ASC;''', (problem,))
            for row in cur:
                result = row[0]
                lang = row[1]
                try:
                    code = row[2].decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        code = row[2].decode('euc-kr')
                    except UnicodeDecodeError:
                        code = code
                user = row[3]
                if 'py' not in lang and 'java' not in lang:
                    continue

                if result == 'correct':
                    answer4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    answer4lang[problem] = answer
                    answers[lang] = answer4lang
                    
                    if user in wrongs.keys():
                        wrongs4team = wrongs[user]
                        if problem in wrongs4team.keys():
                            var, var_dis, wrong_codemap = wrongs4team[problem]
                            v = distance_var2(codemap, wrong_codemap, var)
                            data.append({'subject': subject,
                                         'user': user,
                                         'problem': problem,
                                         'diff': abs(var_dis-v)})
                        if problem in wrongs4team.keys():
                            wrongs4team.pop(problem)
                    if user in wrongs.keys() and len(wrongs[user]) == 0:
                        wrongs.pop(user)

                elif result == 'wrong-answer':
                    answers4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                    if len(answer) > 0:
                        index, answer_codemap = recommend_codemap(answer, codemap)
                        distance = distance_codemap(answer_codemap, codemap)
                        var, var_dis = recommend_var(answer_codemap, codemap)
                        wrongs4team = wrongs.get(user, {})
                        if problem not in wrongs4team.keys():
                            wrongs4team[problem] = (var, var_dis, codemap)
                        wrongs[user] = wrongs4team
        except TypeError:
            continue
df = pd.DataFrame(data)
df

In [ ]:
for subject in ['2019-25155', '2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    total = len(df[df['subject'] == subject])
    pos = len(df.loc[(df['subject'] == subject) & (df['diff'] > 0)])
    nes = len(df.loc[(df['subject'] == subject) & (df['diff'] == 0)])
    print(f'{subject} {total} {pos/total}')

In [ ]:
labels = ['2019-25155', '2020-37777', '2020-25155', '2020-27401A', '2020-27401B']
data = []
for subject in labels:
    total = len(df[df['subject'] == subject])
    pos = len(df.loc[(df['subject'] == subject) & (df['diff'] > 0)])
    nes = len(df.loc[(df['subject'] == subject) & (df['diff'] == 0)])
    data.append(pos/total)
print(data)
fig = plt.figure(figsize=(8*1, 6*1))
fig.set_facecolor('white')
ax = fig.add_subplot()
rect = ax.bar(x=labels, height=data, color='gray')
ax.bar_label(rect, fmt='%0.2f', label_type='edge')
ax.set_ylim((0, 1))
ax.tick_params(axis='x', labelsize='large')
ax.tick_params(axis='y', labelsize='large')
ax.set_xlabel('Subject', fontsize='x-large')
ax.set_ylabel('Accuracy', fontsize='x-large')
fig.savefig('accurary_by_subject.png', bbox_inches='tight')

In [ ]:
data = []
for subject in ['2019-25155', '2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    for grade in ['A', 'B', 'C']:
        total = 0
        pos = 0
        neg = 0
        for user in rank.loc[(rank['subject'] == subject) & (rank['final'] == grade), 'user'].unique():
            for value in df.loc[(df['subject'] == subject) & (df['user'] == user), 'diff']:
                total = total + 1
                if value > 0:
                    pos = pos + 1
                else:
                    neg = neg + 1
        data.append({'subject': subject,
                     'grade': grade,
                     'acc': pos/total,
                    })
acc = pd.DataFrame(data)
acc

In [ ]:
acc['acc'].mean()

In [ ]:
labels = ['2019-25155', '2020-37777', '2020-25155', '2020-27401A', '2020-27401B']

data_a = acc.loc[acc['grade'] == 'A', 'acc'].values
data_b = acc.loc[acc['grade'] == 'B', 'acc'].values
data_c = acc.loc[acc['grade'] == 'C', 'acc'].values

x = np.arange(len(labels))
width = 0.9/3

fig = plt.figure(figsize=(8*1, 6*1))
fig.set_facecolor('white')
ax = fig.add_subplot()
rect1 = ax.bar(x=x-width, height=data_a, width=width, label='Final grade: A')
rect2 = ax.bar(x=x, height=data_b, width=width, label='Final grade: B', tick_label=labels)
rect3 = ax.bar(x=x+width, height=data_c, width=width, label='Final grade: C')
ax.bar_label(rect1, fmt='%0.2f', label_type='edge')
ax.bar_label(rect2, fmt='%0.2f', label_type='edge')
ax.bar_label(rect3, fmt='%0.2f', label_type='edge')
ax.set_ylim((0, 1))
ax.tick_params(axis='x', labelsize='large')
ax.tick_params(axis='y', labelsize='large')
ax.set_xlabel('Subject', fontsize='x-large')
ax.set_ylabel('Accuracy', fontsize='x-large')
ax.legend()
fig.savefig('accurary_by_grade.png', bbox_inches='tight')

# 답안과의 거리 변화

In [ ]:
cnt = 0
data = []

for subject in ['2019-25155']:
    answers = {}
    wrongs = {}
    for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
        try:
            cur.execute('''SELECT dj2019.judging.result, dj2019.submission.langid, dj2019.submission_file.sourcecode,
                                  dj2019.team.name
                           FROM dj2019.submission_file
                           INNER JOIN dj2019.submission ON dj2019.submission_file.submitid = dj2019.submission.submitid
                           INNER JOIN dj2019.judging ON dj2019.submission_file.submitid = dj2019.judging.submitid
                           INNER JOIN dj2019.team ON dj2019.submission.teamid = dj2019.team.teamid
                           WHERE dj2019.submission.probid = ?
                           ORDER BY dj2019.judging.starttime ASC;''', (problem,))
            for row in cur:
                result = row[0]
                lang = row[1]
                try:
                    code = row[2].decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        code = row[2].decode('euc-kr')
                    except UnicodeDecodeError:
                        code = code
                user = row[3]
                if 'py' not in lang and 'java' not in lang:
                    continue

                if result == 'correct':
                    answer4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    answer4lang[problem] = answer
                    answers[lang] = answer4lang
                elif result == 'wrong-answer':
                    answers4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                    if len(answer) > 0:
                        index, answer_codemap = recommend_codemap(answer, codemap)
                        distance = distance_codemap(answer_codemap, codemap)
                        var, var_dis = recommend_var(answer_codemap, codemap)
                        wrongs4team = wrongs.get(user, {})
                        wrongs4team[problem] = (var, var_dis, codemap)
                        wrongs[user] = wrongs4team
                        data.append({'subject': subject,
                                     'var': var,
                                     'problem': problem,
                                     'distance': distance})
        except TypeError:
            continue
for subject in ['2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    answers = {}
    wrongs = {}
    for problem in df_problem.loc[df_problem['subject'] == subject, 'problem']:
        try:
            cur.execute('''SELECT dj2020.judging.result, dj2020.submission.langid, dj2020.submission_file.sourcecode,
                                  dj2020.team.name
                           FROM dj2020.submission_file
                           INNER JOIN dj2020.submission ON dj2020.submission_file.submitid = dj2020.submission.submitid
                           INNER JOIN dj2020.judging ON dj2020.submission_file.submitid = dj2020.judging.submitid
                           INNER JOIN dj2020.team ON dj2020.submission.teamid = dj2020.team.teamid
                           WHERE dj2020.submission.probid = ?
                           ORDER BY dj2020.judging.starttime ASC;''', (problem,))
            for row in cur:
                result = row[0]
                lang = row[1]
                try:
                    code = row[2].decode('utf-8')
                except UnicodeDecodeError:
                    try:
                        code = row[2].decode('euc-kr')
                    except UnicodeDecodeError:
                        code = code
                user = row[3]
                if 'py' not in lang and 'java' not in lang:
                    continue

                if result == 'correct':
                    answer4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                        if len(codemap) > 0:
                            answer.append(codemap)
                    answer4lang[problem] = answer
                    answers[lang] = answer4lang
                elif result == 'wrong-answer':
                    answers4lang = answers.get(lang, {})
                    answer = answer4lang.get(problem, [])
                    if 'py' in lang:
                        codemap = parse_python(code)
                    elif 'java' in lang:
                        codemap = parse_java(code)
                    if len(answer) > 0:
                        index, answer_codemap = recommend_codemap(answer, codemap)
                        distance = distance_codemap(answer_codemap, codemap)
                        var, var_dis = recommend_var(answer_codemap, codemap)
                        wrongs4team = wrongs.get(user, {})
                        wrongs4team[problem] = (var, var_dis, codemap)
                        wrongs[user] = wrongs4team
                        data.append({'subject': subject,
                                     'var': var,
                                     'problem': problem,
                                     'distance': distance})
        except TypeError:
            continue
df = pd.DataFrame(data)
df

In [ ]:
for subject in ['2019-25155', '2020-37777', '2020-25155', '2020-27401A', '2020-27401B']:
    values = df.loc[df['subject'] == subject, 'var'].values
    print(collections.Counter(values))